<a href="https://colab.research.google.com/github/ai10pro/Genshin_Gacha_simulator/blob/main/%E8%AA%B2%E9%A1%8C05_05_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **原神ガチャシュミレーター**

## **概要**
原神というゲームのガチャシュミレーターです。
原神というゲームでは祈願という名前でガチャ要素があり、そのシュミレーションを行うプログラムを作成しました。

機能としては10連シュミレーションを行えるほか、ピックアップキャラクターを個別に選択することができます。


<img src="https://drive.google.com/uc?export=view&id=1ERY06qYVwpGOdrVyugzDbS65EfNKdysx" width=60%>
画像： https://genshin.hoyoverse.com/ja/home


## **操作方法**
基本は実行セル番号の順にセルを実行していきます。

ピックアップキャラクター設定セルでは星4キャラクター3人と星5キャラクター1人をセレクトボックスで選択し、実行してください。

実際にシュミレーションをするときは、メイン実行セルを実行してください。確定演出のあと排出されたキャラクター名（武器名）が表示されます。また、続けてシュミレーションをし続ける場合はメイン実行セルをもう一度実行してください。
ガチャ合計がカウントされ、シュミレーションを行うことができます。

リセットがしたくなった場合はリセットセルを実行し、もう一度はじめと同じ手順でセルを実行してください。


## **制作者のコメント**

原神では90連で必ず星5キャラクターが排出され、また、10連で必ず星4キャラクターが排出されます。また、75連目から6パーセントずつ星5キャラクターのピックアップ確率が上昇します。その機能の実装が難しかったです。

また、このゲームはほかのゲームとは違い、排出するものごとに確率があるのではなく、星3、星4、星5の中で確立を排出キャラクターでさらに分配しています。なので今回は一度、レベルを決める関数を実行し、その値からさらにランダムで排出キャラクターを確定しており、その実装に時間がかかりました。

予定では単発の機能も実装予定でしたが、10連の実装に時間がかかってしまい、最終的に完成させることができませんでした。ほかにも、今まで出た星4キャラクターや星5キャラクターの合計で出た数を表示させる機能の追加もできればよかったです。夏休み期間なども利用して作成してみようと考えています。

## **作成時間**

作成時間：**約21時間**


## **リセットセル**
--実行セル番号①--

ガチャをリセット、ピックアップキャラクターの変更前に必ず実行してください。



In [ ]:
%reset -f
import time
import random as r
# 星3武器のリストとウェイト
l3_arms_names_list = ['弾弓', 'シャープシューターの誓い', '鴉羽の弓','翡玉法珠','龍殺しの英傑譚','魔導緒論','黒纓槍','理屈責め','龍血を浴びた剣','鉄影段平','飛天御剣','黎明の神剣','冷刃']
l3_arms_weight_list = [0.943] * len(l3_arms_names_list)

# 星4キャラと武器のリストとウェイト
l4_names_list = ['凝光', 'ノエル', 'ゴロー', '雲菫', '行秋', 'バーバラ', 'キャンディス', '重雲', 'ガイア', 'ディオナ', 'ミカ', 'レイラ', 'ロサリア', '香菱', 'ベネット', 'アンバー', 'トーマ', '辛炎', '煙緋', 'コレイ', 'カーヴェ', 'ヨォーヨ', '綺良々', '北斗', 'フィッシュル', 'レザー', 'リサ', 'ドリー', '九条裟羅', '久岐忍', 'スクロース', 'ファルザン', '鹿野院平蔵', '早柚', '弓蔵', '祭礼の弓', '絶弦', '西風猟弓', '昭心', '祭礼の断片', '流浪楽章', '西風秘典', '西風長槍', '匣中滅龍', '雨裁', '祭礼の大剣' , '鐘の剣' ,'西風大剣', '匣中龍吟', '祭礼の剣', '笛の剣', '西風剣']
l4_names_weight_list = [0.1/len(l4_names_list)] * len(l4_names_list)

# 星5キャラのリスト
l5_character_names_list = ['アルベド', '荒瀧一斗', '鍾離', '珊瑚宮心海', 'ニィロウ', '神里綾人', '夜蘭', 'タルタリヤ', '甘雨', '申鶴', '神里綾華', 'エウルア', 'クレー', '宵宮', '胡桃', '白朮', 'セノ', '雷電将軍', '八重神子', '放浪者', '鍾離', 'ウェンティ', '楓原万葉']

# 星5恒常キャラのリストとウェイト
l5_reg_names_list = ['ディシア', 'ティナリ' ,'刻晴' ,'モナ' ,'七七', 'ディルック', 'ジン']
l5_reg_names_weight_list = [0.0714] * len(l5_reg_names_list)
l5_reg_names_weight_list.append(0.5)

# レベルごとの値とウェイトのリスト
up = [1, 2, 3]                  # 星3：1,　星4: 2, 星5: 3
weight = [0.943, 0.051, 0.006]

# 合計ガチャ回数カウント
all_sum = 0
sum = 0

# 出力のカラー設定
class Color:
	YELLOW         = '\033[33m'#(文字)黄
	MAGENTA        = '\033[35m'#(文字)マゼンタ
	CYAN           = '\033[36m'#(文字)シアン
	BOLD           = '\033[1m'#太字
	RESET          = '\033[0m'#全てリセット

##  **ピックアップキャラクター設定セル**
--実行セル番号②--

セレクトボックスにて、星4ピックアップキャラクターを3人選択してください。また、星キャラクターを1人選択してください。
選択後、このセルを実行してください。

***※ピックアップキャラクターを変更する場合、セル①をはじめに実行してからキャラクター選択を行い、このセルを実行してください。***

In [ ]:
# 星4ピックアップキャラクターの選択
#@markdown ### **■ 星4ピックアップキャラクター** (4人選択してください)
l4_pickup_character1 = "\u884C\u79CB" #@param ['凝光', 'ノエル', 'ゴロー', '雲菫', '行秋', 'バーバラ', 'キャンディス', '重雲', 'ガイア', 'ディオナ', 'ミカ', 'レイラ', 'ロサリア', '香菱', 'ベネット', 'アンバー', 'トーマ', '辛炎', '煙緋', 'コレイ', 'カーヴェ', 'ヨォーヨ', '綺良々', '北斗', 'フィッシュル', 'レザー', 'リサ', 'ドリー', '九条裟羅', '久岐忍', 'スクロース', 'ファルザン', '鹿野院平蔵', '早柚']
l4_pickup_character2 = "\u96F2\u83EB" #@param ['凝光', 'ノエル', 'ゴロー', '雲菫', '行秋', 'バーバラ', 'キャンディス', '重雲', 'ガイア', 'ディオナ', 'ミカ', 'レイラ', 'ロサリア', '香菱', 'ベネット', 'アンバー', 'トーマ', '辛炎', '煙緋', 'コレイ', 'カーヴェ', 'ヨォーヨ', '綺良々', '北斗', 'フィッシュル', 'レザー', 'リサ', 'ドリー', '九条裟羅', '久岐忍', 'スクロース', 'ファルザン', '鹿野院平蔵', '早柚']
l4_pickup_character3 = "\u30D9\u30CD\u30C3\u30C8" #@param ['凝光', 'ノエル', 'ゴロー', '雲菫', '行秋', 'バーバラ', 'キャンディス', '重雲', 'ガイア', 'ディオナ', 'ミカ', 'レイラ', 'ロサリア', '香菱', 'ベネット', 'アンバー', 'トーマ', '辛炎', '煙緋', 'コレイ', 'カーヴェ', 'ヨォーヨ', '綺良々', '北斗', 'フィッシュル', 'レザー', 'リサ', 'ドリー', '九条裟羅', '久岐忍', 'スクロース', 'ファルザン', '鹿野院平蔵', '早柚']


# 星4ピックアップキャラクターのWeightの変更
l4_names_weight_list[l4_names_list.index(l4_pickup_character1)] = 0.3
l4_names_weight_list[l4_names_list.index(l4_pickup_character2)] = 0.3
l4_names_weight_list[l4_names_list.index(l4_pickup_character3)] = 0.3

# 星5ピックアップキャラクターのガチャリストへの追加
#@markdown ### **■ 星5ピックアップキャラクター** (セレクトボックスから選択してください)
l5_pickup_character = "\u5BB5\u5BAE" #@param ['アルベド', '荒瀧一斗', '鍾離', '珊瑚宮心海', 'ニィロウ', '神里綾人', '夜蘭', 'タルタリヤ', '甘雨', '申鶴', '神里綾華', 'エウルア', 'クレー', '宵宮', '胡桃', '白朮', 'セノ', '雷電将軍', '八重神子', '放浪者', '鍾離', 'ウェンティ', '楓原万葉']
l5_reg_names_list.append(l5_pickup_character)

## 10連をする関数（ten_do()関数）
--実行セル番号③--

10連の処理を行うセルです。
このセルは1度実行するだけでよいです。

In [ ]:
def ten_do(sum,sum_list) :    # sum: ガチャの合計, sum_list:10連ごとに組まれたリスト
  level_list = [0] * 10   # level_listの定義
  for i in range(10) :
    level_list[i] = r.choices(up,weights=weight,k=1)[0]   # level_listにランダムでlevelに割り当てられた数字を入れる
    sum = sum + 1                                         # ガチャ合計＋1
    if sum >= 75 and  level_list.count(3) == 0:           # ガチャ合計が75以上かつ、星5排出がない場合
      weight[2] = weight[2] + 0.06                        # 星5の排出確立を6パーセント増やす
    else :
      weight[2] = 0.006                                   # そうでないときは排出確立をもとに戻す
  l2_count = level_list.count(2)                      # level2の数をカウント
  l3_count = level_list.count(3)                      # level3の数をカウント

  # ガチャの最低保証の条件確保
  if sum >= 90 and l3_count == 0 and l2_count == 0:   # 90連以上の回で星5と星4の排出がないとき
    level_list[9] = 3                                 # 出力の一番最後を星5にする
    level_list[8] = 2                                 # 出力の最後から2番目を星4にする
    sum = 0                                           # 合計値をリセットする
  elif sum >= 90 and l3_count == 0:                   # 90連以上の回で星5の排出がないとき
    level_list[9] = 3                                 # 出力の一番最後を星5にする
    sum = 0                                           # 合計値をリセットする
  elif l3_count >= 1 :                                # 星5が1つ以上ならば合計を0にする
    sum = 0
  elif l2_count == 0 :                                # 星4の最低保証の実行
    level_list[9] = 2

  # ↓リストを出力するプログラム
  cnt = sum_list.count(0)           # sum_listの0の数を取得
  if sum % 10 == 0 :                # sumが10の倍数の時、ガチャ合計とレベルリストを出力
    return sum,level_list           # 10ごとにリストを作っているのでぴったり10の倍数ではsumと完成したlevel_listの出力
  else :
    sum = sum - 9                              # sumの値を取得したときの値にリセット
    start = 9 - cnt                            # sum_listをどこから始めるのか
    for i in range(cnt) :
      sum_list[start + i] = level_list[i]
      sum = sum + 1
    level_list2 = [0] * 10
    for i in range(9 - cnt) :
      level_list2[i] = level_list[cnt + 1]
      sum = sum + 1
    # return(sum,sum_list,level_list2)
    return sum,level_list,sum_list,level_list2

## キャラ制定関数（gacha_output()関数）
--実行セル番号④--

このセルでは実際にガチャを行う関数で得られたリストから、キャラクターをランダムピックしていく関数です。

シュミレーションを行う場合、1度実行するだけでよいです。

In [ ]:
def gacha_output(result_list) :       # result_list:レベルが決まったリスト
  gacha_list = [0] * 10               # gacha_listの作成
  cnt = result_list.count(0)          # result_listのゼロの数より何回キャラ制定をするのかのカウント
  for i in range (10 - cnt) :
    if result_list[i] == 1 :          # result_listが1の時
      # print(f'{result[1][i]} : 星3')
      gacha_list[i] = r.choices(l3_arms_names_list,weights=l3_arms_weight_list,k=1)[0]  # 星3キャラの制定
    elif result_list[i] == 2 :       # result_listが2の時
      # print(f'{result[1][i]} : 星4')
      gacha_list[i] = Color.MAGENTA + r.choices(l4_names_list,weights=l4_names_weight_list,k=1)[0] + "    ★4" + Color.RESET  # 星4キャラ制定 + 出力文字色の変更
    elif result_list[i] == 3 :       # result_listが3の時
      # print(f'{result[1][i]} : 星5')
      gacha_list[i] = Color.YELLOW + r.choices(l5_reg_names_list,weights=l5_reg_names_weight_list,k=1)[0] + "   ☆" + Color.RESET # 星5キャラ制定 + 出力文字色の変更

  return(gacha_list)

## 確定演出出力関数（output()関数）
--実行セル番号⑤--

この関数は確定演出を行う関数です。
星5が出たときは黄色のラインが表示、星4の時、紫、それ以外の時は水色のラインを出力します。
シュミレーションの際は1度のみの実行で動作します。

In [ ]:
def output(sum,gacha_output_list,level_list):
  cnt_2 = level_list.count(2)     # 星4の数のカウント
  cnt_3 = level_list.count(3)     # 星5の数のカウント
  for i in range (15) :
    if cnt_3 >= 1 and i >= 7 :                            # 星5が1つ以上あるとき
      print(f'{Color.YELLOW}---{Color.RESET}',end = " ")  # 線の色をyellowにする
    elif cnt_2 >= 1 and i >= 7 :                          # 星4が1つ以上あるとき
      print(f'{Color.MAGENTA}---{Color.RESET}',end = " ") # 線の色をMagentaにする
    else :                                                # 以上のどれでもないとき
      print(f'{Color.CYAN}---{Color.RESET}',end=" ")      # 線の色をCyanにする
    time.sleep(0.2)                                       # 0.2秒処理を止める

  print() # 改行

## メイン実行セル

シュミレーションをしたい回数繰り返し実行してください。

**※1実行中に再びセルの実行はできません。また、止まったように見えるかもしれませんが排出が10個されたのち、もう一度セルの実行を行えます。**

**※2関数の中で複雑な動作を行っているため出力が不安定です。止まったように見えるかもしれませんが焦らず実行してください。**

In [ ]:
gacha_level_list = [0] * 10

print(f'{all_sum + 10} 連目')
all_sum = all_sum + 10
print()
print(f'星5排出後から{sum + 10} 連目')
result = ten_do(sum,gacha_level_list)         # resultにten_do関数の結果の代入
sum = result[0]                               # sum にten_do後の合計値
result_list = result[1]                       # result_list にten_do後のリスト
gacha_output_list = gacha_output(result_list) # ガチャのレアリティのリストからキャラクターの制定
output(sum,gacha_output_list,result_list)     # 確定演出関数の実行

for up_n in range(len(gacha_output_list)) :   # 出力されたキャラクターの数の分繰り返す
  print(f'{gacha_output_list[up_n]}')         # 0.6秒ごとに出力する。
  time.sleep(0.6)
# print(f'{sum}:{gacha_output_list}')



130 連目

星5排出後から40 連目
--- --- --- --- --- --- --- --- --- --- --- --- --- --- --- 
弾弓
龍殺しの英傑譚
魔導緒論
飛天御剣
黒纓槍
魔導緒論
鴉羽の弓
理屈責め
翡玉法珠
ベネット    ★4
